# Vehicle Price Prediction - Big Data Project Milestone 2   
https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fwww.geeksforgeeks.org%2Fhow-to-import-kaggle-datasets-directly-into-google-colab%2F

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=dc6a25156eab7d2d6a3a38921738c2caeef505ceca2eaea06f44ccfedfdf75a3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
import pyspark

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import warnings
from sklearn.metrics import mean_absolute_error


In [ ]:
od.download("https://www.kaggle.com/datasets/syedanwarafridi/vehicle-sales-data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: faridayoussef
Your Kaggle Key: ··········


100%|██████████| 18.8M/18.8M [00:01<00:00, 13.8MB/s]


In [ ]:
df = pd.read_csv("/content/vehicle-sales-data/car_prices.csv")
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


# Handling Missing Values

In [ ]:
missing_values = df.isnull().sum()
total_values = df.shape[0]
percentage_missing = (missing_values / total_values) * 100
columns_with_missing = missing_values[missing_values > 0]

missing_data_table = []
for col in columns_with_missing.index:
    missing_data_table.append([col, missing_values[col], f"{percentage_missing[col]:.2f}%"])

print(tabulate(missing_data_table, headers=['Column', 'Missing Values', 'Percentage']))


Column          Missing Values  Percentage
------------  ----------------  ------------
make                     10301  1.84%
model                    10399  1.86%
trim                     10651  1.91%
body                     13195  2.36%
transmission             65352  11.69%
vin                          4  0.00%
condition                11820  2.12%
odometer                    94  0.02%
color                      749  0.13%
interior                   749  0.13%
mmr                         38  0.01%
sellingprice                12  0.00%
saledate                    12  0.00%


In [ ]:
# Drop rows with missing values for all columns except 'transmission'
df_cleaned = df.copy()
cols_to_check = ['make', 'model', 'trim', 'body', 'vin', 'condition', 'odometer', 'color', 'interior', 'mmr', 'sellingprice', 'saledate']
df_cleaned.dropna(subset=cols_to_check, inplace=True)

# Check if there are any remaining missing values
missing_values_after_drop = df_cleaned.isnull().sum()
print("Remaining missing values after dropping rows:")
print(missing_values_after_drop[missing_values_after_drop > 0])


Remaining missing values after dropping rows:
transmission    61323
dtype: int64


## KNN Imputation

In [ ]:
# KNN imputation for transmission with K = 5

df_imputed = df_cleaned.copy()
encoder = OrdinalEncoder()
df_imputed[['transmission']] = encoder.fit_transform(df_imputed[['transmission']])

imputer = KNNImputer(n_neighbors=5)
transmission_col = 'transmission'
transmission_imputed = imputer.fit_transform(df_imputed[[transmission_col]])
df_imputed[transmission_col] = transmission_imputed

# Decode 'transmission' column back to original values
df_imputed[['transmission']] = encoder.inverse_transform(df_imputed[['transmission']])

missing_values_after_imputation = df_imputed.isnull().sum()
print("Remaining missing values after imputation:")
print(missing_values_after_imputation[missing_values_after_imputation > 0])

In [ ]:
df_imputed.isnull().sum()

# Standardization

In [ ]:
numeric_cols = ['condition','odometer', 'mmr', 'sellingprice','year']
scaler = StandardScaler()
df_imputed[numeric_cols] = scaler.fit_transform(df_imputed[numeric_cols])

# Feature Engineering

In [ ]:
# Drop VIN because it has 525638 unique values

unique_vin_count = df_imputed['vin'].nunique()
print("Total number of unique VINs:", unique_vin_count)
df_imputed.drop('vin', axis=1, inplace=True)
df_imputed.drop('vin', axis=1, inplace=True)

In [ ]:
# Convert 'saledate' column to datetime format
df_imputed['saledate'] = pd.to_datetime(df_imputed['saledate'], utc=True, errors='coerce')

# Extract day of the week as 0-based index (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
df_imputed['day_of_week'] = df_imputed['saledate'].dt.dayofweek

# Extract month as 1-based index (1 = January, 2 = February, ..., 12 = December)
df_imputed['month'] = df_imputed['saledate'].dt.month

print(df_imputed[['saledate', 'day_of_week', 'month']])

# VIF - Multicollinearity

In [ ]:
X = df_imputed[['condition','odometer', 'mmr','year']]
X_with_const = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_const.values, i) for i in range(X_with_const.shape[1])]
print(vif_data)


# Outliers Detection

In [ ]:

columns_to_plot = ['condition', 'odometer', 'mmr', 'sellingprice']
plt.figure(figsize=(15, 8))
for i, col in enumerate(columns_to_plot, 1):
    plt.subplot(2, 3, i)
    plt.boxplot(df_imputed[col].dropna())
    plt.title(col)
plt.tight_layout()
plt.show()


In [ ]:
# Define thresholds
odometer_threshold = 12
sellingprice_threshold = 17

# Remove outliers
df_filtered = df_imputed[(df_imputed['odometer'] <= odometer_threshold) & (df_imputed['sellingprice'] <= sellingprice_threshold)]
print(df_filtered)


In [ ]:
# Boxplots after outlier removal

columns_to_plot = ['condition', 'odometer', 'mmr', 'sellingprice']
plt.figure(figsize=(15, 8))
for i, col in enumerate(columns_to_plot, 1):
    plt.subplot(2, 3, i)
    plt.boxplot(df_filtered[col].dropna())
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
df_filtered.to_csv('df_filtered.csv', index=False)

# Fitting Models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Big Data Project/df_filtered.csv'

df = pd.read_csv(file_path)


In [ ]:
df.drop('seller', axis=1, inplace=True)


In [ ]:

#columns_to_encode = ['make', 'model', 'trim', 'body', 'transmission', 'state', 'color', 'interior', 'seller']
#df_encoded = pd.get_dummies(df, columns=columns_to_encode)
#print(df_encoded)


## Linear Regression

In [2]:
!pip3 install http://h2o-release.s3.amazonaws.com/h2o/rel-zz_kurka/2/Python/h2o-3.40.0.2-py2.py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 MB 31.9 MB/s eta 0:00:00


In [4]:
import h2o
from h2o.estimators import H2OGeneralizedLinearEstimator
h2o.init()

h2o_df = h2o.H2OFrame(df)

feature_columns = ['year', 'make', 'model', 'odometer', 'condition', 'trim', 'body', 'transmission', 'mmr']
target_column = 'sellingprice'

train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

for col in ['make', 'model', 'trim', 'body', 'transmission']:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

# Initialize the H2O Linear Regression model
model = H2OGeneralizedLinearEstimator(family="gaussian")

model.train(x=feature_columns, y=target_column, training_frame=train)

predictions = model.predict(test)

predictions_df = predictions.as_data_frame()

# Evaluate the model
mse = model.mse()
r2 = model.r2()
mae = model.mae()
rmse = model.rmse()

print(f'Linear Regression MSE: {mse}')
print(f'Linear Regression R² score: {r2}')
print(f'Linear Regression MAE: {mae}')
print(f'Linear Regression RMSE: {rmse}')

# Shutdown H2O
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpp0u8g_yk
  JVM stdout: /tmp/tmpp0u8g_yk/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpp0u8g_yk/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,"1 year, 1 month and 27 days"
H2O_cluster_name:,H2O_from_python_unknownUser_aoa48h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.75 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


NameError: name 'df' is not defined

## Gradient Boosting

In [ ]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
h2o.init()

# Assuming df is your pandas DataFrame
h2o_df = h2o.H2OFrame(df)

# Define feature columns and target column
feature_columns = ['year', 'make', 'model', 'odometer', 'condition', 'trim', 'body', 'transmission', 'mmr']
target_column = 'sellingprice'

# Split the H2OFrame into train and test sets
train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

# Convert categorical columns to factors
for col in ['make', 'model', 'trim', 'body', 'transmission']:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

# Initialize the H2O Gradient Boosting model for regression
model = H2OGradientBoostingEstimator(ntrees=100, max_depth=5, seed=42)

# Train the model
model.train(x=feature_columns, y=target_column, training_frame=train)

# Make predictions
predictions = model.predict(test)

# Convert predictions to a pandas DataFrame for evaluation
predictions_df = predictions.as_data_frame()

# Evaluate the model
mse = model.mse()
r2 = model.r2()
mae = model.mae()
rmse = model.rmse()

print(f'Gradient Boosting Regression MSE: {mse}')
print(f'Gradient Boosting Regression R² score: {r2}')
print(f'Gradient Boosting Regression MAE: {mae}')
print(f'Gradient Boosting Regression RMSE: {rmse}')

# Shutdown H2O
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_z9sgttk
  JVM stdout: /tmp/tmp_z9sgttk/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_z9sgttk/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,"1 year, 1 month and 12 days"
H2O_cluster_name:,H2O_from_python_unknownUser_gb2ux5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.75 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Gradient Boosting Regression MSE: 0.020728907996993458
Gradient Boosting Regression R² score: 0.9792193165308428
Gradient Boosting Regression MAE: 0.09296625434318392
Gradient Boosting Regression RMSE: 0.14397537288367568
H2O session _sid_8e25 closed.


## Random Forest Regressor

In [ ]:
import h2o
from h2o.estimators import H2ORandomForestEstimator
h2o.init()

# Assuming df is your pandas DataFrame
h2o_df = h2o.H2OFrame(df)

# Define feature columns and target column
feature_columns = ['year', 'make', 'model', 'odometer', 'condition', 'trim', 'body', 'transmission', 'mmr']
target_column = 'sellingprice'

# Split the H2OFrame into train and test sets
train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

# Convert categorical columns to factors
for col in ['make', 'model', 'trim', 'body', 'transmission']:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

# Initialize the H2O Random Forest model for regression
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, seed=42)

# Train the model
model.train(x=feature_columns, y=target_column, training_frame=train)

# Make predictions
predictions = model.predict(test)

# Convert predictions to a pandas DataFrame for evaluation
predictions_df = predictions.as_data_frame()

# Evaluate the model
mse = model.mse()
r2 = model.r2()
mae = model.mae()
rmse = model.rmse()

print(f'Decision Tree Regression MSE: {mse}')
print(f'Decision Tree Regression R² score: {r2}')
print(f'Decision Tree Regression MAE: {mae}')
print(f'Decision Tree Regression RMSE: {rmse}')

# Shutdown H2O
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpch7b7wc9
  JVM stdout: /tmp/tmpch7b7wc9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpch7b7wc9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,"1 year, 1 month and 12 days"
H2O_cluster_name:,H2O_from_python_unknownUser_5g2cwd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.75 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Decision Tree Regression MSE: 0.02509401706107328
Decision Tree Regression R² score: 0.9748433045488248
Decision Tree Regression MAE: 0.09823908518777644
Decision Tree Regression RMSE: 0.15841091206439437
H2O session _sid_9252 closed.


## XGBoost

In [ ]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
h2o.init()

# Assuming df is your pandas DataFrame
h2o_df = h2o.H2OFrame(df)

# Define feature columns and target column
feature_columns = ['year', 'make', 'model', 'odometer', 'condition', 'trim', 'body', 'transmission', 'mmr']
target_column = 'sellingprice'

# Split the H2OFrame into train and test sets
train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

# Convert categorical columns to factors
for col in ['make', 'model', 'trim', 'body', 'transmission']:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

# Initialize the H2O XGBoost model for regression
model = H2OXGBoostEstimator(ntrees=100, max_depth=5, seed=42)

# Train the model
model.train(x=feature_columns, y=target_column, training_frame=train)

# Make predictions
predictions = model.predict(test)

# Convert predictions to a pandas DataFrame for evaluation
predictions_df = predictions.as_data_frame()

# Evaluate the model
mse = model.mse()
r2 = model.r2()
mae = model.mae()
rmse = model.rmse()

print(f'XGBoost Regression MSE: {mse}')
print(f'XGBoost Regression R² score: {r2}')
print(f'XGBoost Regression MAE: {mae}')
print(f'XGBoost Regression RMSE: {rmse}')

# Shutdown H2O
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,1 min 38 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,"1 year, 1 month and 12 days"
H2O_cluster_name:,H2O_from_python_unknownUser_11hac9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.70 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
XGBoost Regression MSE: 0.02109661374759605
XGBoost Regression R² score: 0.9788506923460006
XGBoost Regression MAE: 0.09460924500897631
XGBoost Regression RMSE: 0.14524673403418079
H2O session _sid_98fc closed.


## Adaboost Regression

In [ ]:
import h2o
from h2o.estimators import H2OSupportVectorMachineEstimator
h2o.init()


# Assuming df is your pandas DataFrame
h2o_df = h2o.H2OFrame(df)

# Define feature columns and target column
feature_columns = ['year', 'make', 'model', 'odometer', 'condition', 'trim', 'body', 'transmission', 'mmr']
target_column = 'sellingprice'

# Split the H2OFrame into train and test sets
train, test = h2o_df.split_frame(ratios=[0.8], seed=42)

# Convert categorical columns to factors
for col in ['make', 'model', 'trim', 'body', 'transmission']:
    train[col] = train[col].asfactor()
    test[col] = test[col].asfactor()

# Initialize the H2O Gradient Boosting model using Gaussian Distribution for regression (AdaBoost)
model = H2OGradientBoostingEstimator(distribution='gaussian', ntrees=100, max_depth=5, seed=42)

# Train the model
model.train(x=feature_columns, y=target_column, training_frame=train)

# Make predictions
predictions = model.predict(test)

# Convert predictions to a pandas DataFrame for evaluation
predictions_df = predictions.as_data_frame()

# Evaluate the model
mse = model.mse()
r2 = model.r2()
mae = model.mae()
rmse = model.rmse()

print(f'AdaBoost Regression MSE: {mse}')
print(f'AdaBoost Regression R² score: {r2}')
print(f'AdaBoost Regression MAE: {mae}')
print(f'AdaBoost Regression RMSE: {rmse}')

# Shutdown H2O
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,3 mins 43 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,"1 year, 1 month and 12 days"
H2O_cluster_name:,H2O_from_python_unknownUser_fff2ye
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12.64 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
AdaBoost Regression MSE: 0.020728907996993458
AdaBoost Regression R² score: 0.9792193165308428
AdaBoost Regression MAE: 0.09296625434318392
AdaBoost Regression RMSE: 0.14397537288367568
H2O session _sid_b736 closed.


## Decision Tree Regression

In [ ]:
y = df['sellingprice']
feature_columns = ['year', 'make', 'model', 'odometer', 'condition','trim','body','transmission','mmr']
X = df[feature_columns]

numerical_features = ['year', 'odometer', 'condition','mmr']
categorical_features = ['make', 'model']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree_pipeline.fit(X_train, y_train)

y_pred = tree_pipeline.predict(X_test)
print(y_pred)
print(f'Decision Tree MSE: {mean_squared_error(y_test, y_pred)}')
print(f'Decision Tree R² score: {r2_score(y_test, y_pred)}')
print(f'Decision Tree MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'Decision Tree RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')


[-0.19228386  0.08682789 -1.07096899 ...  0.11784031  0.51066425
  0.32458975]
Decision Tree MSE: 0.04510958047369386
Decision Tree R² score: 0.9543780483353344


In [ ]:
print(f'Decision Tree MAE: {mean_absolute_error(y_test, y_pred)}')

Decision Tree MAE: 0.13509189275255187


In [ ]:
print(f'Decision Tree RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')

Decision Tree RMSE: 0.2123901609625405
